# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [4]:
stocks=pd.read_csv('sp_500_stocks.csv')
from secretstest import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [35]:
symbol='aapl'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data=requests.get(api_url).json()
print(data)

{'avgTotalVolume': 70144611, 'calculationPrice': 'tops', 'change': -0.44, 'changePercent': -0.00299, 'close': 0, 'closeSource': 'iffialoc', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 157.6, 'iexAskSize': 109, 'iexBidPrice': 149.58, 'iexBidSize': 152, 'iexClose': 157.4, 'iexCloseTime': 1656910915856, 'iexLastUpdated': 1703868417210, 'iexMarketPercent': 0.010095150370142702, 'iexOpen': 154.82, 'iexOpenTime': 1646185500309, 'iexRealtimePrice': 156.22, 'iexRealtimeSize': 103, 'iexVolume': 442067, 'lastTradeTime': 1688428397516, 'latestPrice': 154.91, 'latestSource': 'IEX real time price', 'latestTime': '2:57:52 PM', 'latestUpdate': 1683808945927, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2500298798899, 'oddLotDelaye

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [36]:
price=data['latestPrice']
pe_ratio=data['peRatio']
pe_ratio

13.9

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [37]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [38]:
final_dataframe=pd.DataFrame(columns=my_columns)
final_dataframe
for symbol_string in symbol_strings:
    batch_url=f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    
    data=requests.get(batch_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe=final_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],
            index=my_columns
        ),
            ignore_index=True
        )
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,160.500,49.71,N/A
1,AAL,21.910,-4.48,N/A
2,AAP,235.570,25.69,N/A
3,AAPL,153.090,13.7,N/A
4,ABBV,122.270,28.8,N/A
5,ABC,132.770,-7.11,N/A
6,ABMD,366.734,116.04,N/A
7,ABT,130.000,36.24,N/A
8,ACN,380.000,41.36,N/A
9,ADBE,668.750,56.01,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [39]:
final_dataframe.sort_values("Price-to-Earnings Ratio",inplace=True)
final_dataframe=final_dataframe[final_dataframe['Price-to-Earnings Ratio']>0]
final_dataframe=final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop("index",axis=1,inplace=True)

In [40]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,744.214,3.37,N/A
1,EBAY,77.860,4.26,N/A
2,MPC,67.820,4.7,N/A
3,LYB,94.000,5.6,N/A
4,PRU,114.330,6.27,N/A
5,COF,161.980,6.28,N/A
6,ZION,66.620,6.53,N/A
7,VIAC,37.790,7,N/A
8,WHR,219.680,7.02,N/A
9,GS,428.270,7.04,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [41]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [42]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [46]:
position_size=float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row,'Number of Shares to Buy']=math.floor(position_size/final_dataframe.loc[row,'Price'])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,744.214,3.37,26
1,EBAY,77.860,4.26,256
2,MPC,67.820,4.7,294
3,LYB,94.000,5.6,212
4,PRU,114.330,6.27,174
5,COF,161.980,6.28,123
6,ZION,66.620,6.53,300
7,VIAC,37.790,7,529
8,WHR,219.680,7.02,91
9,GS,428.270,7.04,46


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [53]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

print(ev_to_gross_profit)

8.540129852200401


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [59]:
rv_columns=[
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score',
]

rv_dataframe=pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe=rv_dataframe.append(
        pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index=rv_columns),
            ignore_index=True,
        )


In [60]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,161.600,N/A,50.82,N/A,9.97,N/A,7.84,N/A,32.307798,N/A,15.099898,N/A,N/A
1,AAL,22.493,N/A,-4.36,N/A,-1.8,N/A,0.5679,N/A,-7.730760,N/A,1.571837,N/A,N/A
2,AAP,237.220,N/A,25.36,N/A,4.41,N/A,1.38,N/A,12.885616,N/A,3.090371,N/A,N/A
3,AAPL,157.297,N/A,13.51,N/A,20,N/A,3.54,N/A,11.267076,N/A,8.729175,N/A,N/A
4,ABBV,119.480,N/A,28.45,N/A,15.31,N/A,3.91,N/A,10.087580,N/A,7.117989,N/A,N/A
5,ABC,132.680,N/A,-7.11,N/A,727.76,N/A,0.1338,N/A,9.823547,N/A,5.026127,N/A,N/A
6,ABMD,372.110,N/A,113.19,N/A,12.44,N/A,17.19,N/A,54.469983,N/A,19.488719,N/A,N/A
7,ABT,127.276,N/A,36.33,N/A,6.79,N/A,5.84,N/A,25.888582,N/A,10.821976,N/A,N/A
8,ACN,384.510,N/A,41.05,N/A,12.64,N/A,4.78,N/A,24.538090,N/A,13.974644,N/A,N/A
9,ADBE,694.900,N/A,55.63,N/A,23.35,N/A,22.21,N/A,53.607878,N/A,23.166418,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [61]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)] #broken data

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,190.200,N/A,61.46,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,302.620,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,291.050,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.85,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,26.010,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.580,N/A,14.8,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,59.790,N/A,33.43,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,39.270,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2992.830,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
325,MXIM,103.600,N/A,35.19,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [62]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [63]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [67]:
from scipy.stats import percentileofscore as score
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row,metrics[metric]] =score(rv_dataframe[metric],rv_dataframe.loc[row,metric])
        #rv_dataframe.loc[row,metrics[metric]] =stats.percentileofscore(rv_dataframe[metric],rv_dataframe.loc[row,metric])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,161.600,N/A,50.82,81.7822,9.970000,79.604,7.84000,80.8911,32.307798,88.3168,15.099898,83.1683,N/A
1,AAL,22.493,N/A,-4.36,6.93069,-1.800000,5.14851,0.56790,4.15842,-7.730760,2.77228,1.571837,6.53465,N/A
2,AAP,237.220,N/A,25.36,46.6337,4.410000,54.8515,1.38000,18.5149,12.885616,40.198,3.090371,14.4554,N/A
3,AAPL,157.297,N/A,13.51,24.3564,20.000000,92.0792,3.54000,49.604,11.267076,31.0891,8.729175,58.6139,N/A
4,ABBV,119.480,N/A,28.45,55.0495,15.310000,89.3069,3.91000,54.9505,10.087580,26.7327,7.117989,44.1584,N/A
5,ABC,132.680,N/A,-7.11,6.13861,727.760000,100,0.13380,0.594059,9.823547,24.9505,5.026127,29.703,N/A
6,ABMD,372.110,N/A,113.19,95.8416,12.440000,84.1584,17.19000,94.8515,54.469983,95.6436,19.488719,91.0891,N/A
7,ABT,127.276,N/A,36.33,68.7129,6.790000,71.6832,5.84000,72.8713,25.888582,79.2079,10.821976,70.297,N/A
8,ACN,384.510,N/A,41.05,73.8614,12.640000,84.5545,4.78000,62.8713,24.538090,75.6436,13.974644,79.802,N/A
9,ADBE,694.900,N/A,55.63,84.7525,23.350000,93.6634,22.21000,98.0198,53.607878,95.2475,23.166418,95.4455,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [86]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles=[]
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row,metrics[metric]]/100)
    rv_dataframe.loc[row,'RV Score']=mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.796,1865.95,7.99,11.2871,0.5039,5.34653,0.4161,2.37624,3.350061,3.76238,0.404359,0.792079,0.0471287
1,AAL,22.493,2222.91,-4.36,6.93069,-1.8000,5.14851,0.5679,4.15842,-7.730760,2.77228,1.571837,6.53465,0.0510891
2,PRU,111.320,449.156,6.37,8.91089,0.7033,5.94059,0.6737,5.74257,4.946930,5.34653,0.672586,1.18812,0.0542574
3,ALL,117.210,426.585,9.66,14.6535,1.3400,12.0792,0.7158,6.33663,2.191530,3.36634,0.688457,1.38614,0.0756436
4,BA,233.830,213.831,-16.12,4.9505,-9.1000,4.35644,2.0000,30.297,-34.269195,0.792079,-1394.202685,0.19802,0.0811881
5,L,59.040,846.883,9.72,15.0495,0.8147,6.93069,1.0029,11.8812,5.181659,5.94059,0.980085,2.77228,0.0851485
6,MCK,229.190,218.16,-7.53,5.74257,-60.9000,1.58416,0.1370,0.792079,8.889709,21.1881,3.035422,13.8614,0.0863366
7,RE,273.000,183.15,8.28,12.2772,1.0700,8.31683,0.9916,11.4851,7.062748,12.2772,0.950959,2.57426,0.0938614
8,LNC,72.620,688.516,10.38,17.0297,0.6428,5.74257,0.6922,6.13861,7.973678,17.0297,0.695488,1.58416,0.0950495
9,MET,65.150,767.46,13.35,23.4653,0.8294,7.12871,0.8150,8.51485,5.844666,6.73267,0.794671,1.78218,0.0952475


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [87]:
rv_dataframe.sort_values('RV Score',ascending=True,inplace=True)
rv_dataframe=rv_dataframe[:50]
rv_dataframe.reset_index(drop=True,inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.796,1865.95,7.99,11.2871,0.5039,5.34653,0.4161,2.37624,3.350061,3.76238,0.404359,0.792079,0.0471287
1,AAL,22.493,2222.91,-4.36,6.93069,-1.8000,5.14851,0.5679,4.15842,-7.730760,2.77228,1.571837,6.53465,0.0510891
2,PRU,111.320,449.156,6.37,8.91089,0.7033,5.94059,0.6737,5.74257,4.946930,5.34653,0.672586,1.18812,0.0542574
3,ALL,117.210,426.585,9.66,14.6535,1.3400,12.0792,0.7158,6.33663,2.191530,3.36634,0.688457,1.38614,0.0756436
4,BA,233.830,213.831,-16.12,4.9505,-9.1000,4.35644,2.0000,30.297,-34.269195,0.792079,-1394.202685,0.19802,0.0811881
5,L,59.040,846.883,9.72,15.0495,0.8147,6.93069,1.0029,11.8812,5.181659,5.94059,0.980085,2.77228,0.0851485
6,MCK,229.190,218.16,-7.53,5.74257,-60.9000,1.58416,0.1370,0.792079,8.889709,21.1881,3.035422,13.8614,0.0863366
7,RE,273.000,183.15,8.28,12.2772,1.0700,8.31683,0.9916,11.4851,7.062748,12.2772,0.950959,2.57426,0.0938614
8,LNC,72.620,688.516,10.38,17.0297,0.6428,5.74257,0.6922,6.13861,7.973678,17.0297,0.695488,1.58416,0.0950495
9,MET,65.150,767.46,13.35,23.4653,0.8294,7.12871,0.8150,8.51485,5.844666,6.73267,0.794671,1.78218,0.0952475


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [94]:
portfolio_input()

Enter the value of your portfolio:2500000


In [95]:
position_size=float(portfolio_size)/len(rv_dataframe.index)
position_size==2500000/50
for row in rv_dataframe.index:
    rv_dataframe.loc[row,'Number of Shares to Buy']=position_size/rv_dataframe.loc[row,'Price']
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.796,1865.95,7.99,11.2871,0.5039,5.34653,0.4161,2.37624,3.350061,3.76238,0.404359,0.792079,0.0471287
1,AAL,22.493,2222.91,-4.36,6.93069,-1.8000,5.14851,0.5679,4.15842,-7.730760,2.77228,1.571837,6.53465,0.0510891
2,PRU,111.320,449.156,6.37,8.91089,0.7033,5.94059,0.6737,5.74257,4.946930,5.34653,0.672586,1.18812,0.0542574
3,ALL,117.210,426.585,9.66,14.6535,1.3400,12.0792,0.7158,6.33663,2.191530,3.36634,0.688457,1.38614,0.0756436
4,BA,233.830,213.831,-16.12,4.9505,-9.1000,4.35644,2.0000,30.297,-34.269195,0.792079,-1394.202685,0.19802,0.0811881
5,L,59.040,846.883,9.72,15.0495,0.8147,6.93069,1.0029,11.8812,5.181659,5.94059,0.980085,2.77228,0.0851485
6,MCK,229.190,218.16,-7.53,5.74257,-60.9000,1.58416,0.1370,0.792079,8.889709,21.1881,3.035422,13.8614,0.0863366
7,RE,273.000,183.15,8.28,12.2772,1.0700,8.31683,0.9916,11.4851,7.062748,12.2772,0.950959,2.57426,0.0938614
8,LNC,72.620,688.516,10.38,17.0297,0.6428,5.74257,0.6922,6.13861,7.973678,17.0297,0.695488,1.58416,0.0950495
9,MET,65.150,767.46,13.35,23.4653,0.8294,7.12871,0.8150,8.51485,5.844666,6.73267,0.794671,1.78218,0.0952475


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [96]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [97]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [98]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [99]:
writer.save()